In [ ]:
## https://docs.python.org/3/library/threading.html

In [4]:
import threading
from threading import Thread

import logging

FORMAT = '%(asctime)-15s %(threadName)s: %(message)s'

logging.basicConfig(level=logging.INFO, format=FORMAT)
logger = logging.getLogger(__name__)

In [5]:
def where_am_i():
    print("Now I am running %s the main thread" % ('IN ' if threading.current_thread() == threading.main_thread() else 'NOT IN '))

def heavy_work():
    int_log = logging.getLogger(__name__)
    where_am_i()
    print("Performing some actions ... ")
    int_log.info("Exit the thead")
    
    
where_am_i()
    
t = Thread(name='MyThread', target=heavy_work)
t.start()




Now I am running IN  the main thread
Now I am running NOT IN  the main thread
Performing some actions ... 


2018-09-05 11:35:48,979 MyThread: Exit the thead


In [6]:

class MyThread(Thread):
    def __init__(self):
        super().__init__()
        
    def run(self):
        print("This is actually what this thread is doing...")
        
        
mt = MyThread()
mt.start()

This is actually what this thread is doing...


In [7]:

class SimpleThread():
    def work(self, a):
        print(a)
        
st = SimpleThread()
        
Thread(target=st.work, args=('Hello from thread',)).start()



Hello from thread


In [8]:
import threading
import time
from threading import Lock

## A bus seats booking
## Actions: Book, pay, confirm


def remaining_seats(initial):
    if initial <= 0:
        return None
    
    number = 1
    while number <= initial:
        yield ("#{0:0" + str(len(str(initial))) + "d}").format(number)
        number += 1
        
    return None

class Cashier(Thread):    
    def __init__(self, lock, seats, cashier_name):
        super().__init__(name=cashier_name)
        self.lock = lock
        self.seats = seats
        
        
    def run(self):

        while True:
            self.lock.acquire()
            try:
                ticket = next(bus_seats)
                logger.info("Solding out a ticket %s ..." % ticket)
                time.sleep(0.005)
                logger.info("Ticket %s is sold now" % ticket)
            except StopIteration as sierr:
                logger.info("No more tickets left. The cashier is closing now ...")
                return;
            finally:
                self.lock.release()

            time.sleep(0.001)
        
        
bus_seats = remaining_seats(20)
booking_lock = Lock()

c1 = Cashier(booking_lock, bus_seats, "Cashier #1")
c2 = Cashier(booking_lock, bus_seats, "Cashier #2")
c3 = Cashier(booking_lock, bus_seats, "Cashier #3")

c1.start()
c2.start()
c3.start()

2018-09-05 11:35:55,093 Cashier #1: Solding out a ticket #01 ...
2018-09-05 11:35:55,102 Cashier #1: Ticket #01 is sold now
2018-09-05 11:35:55,102 Cashier #2: Solding out a ticket #02 ...
2018-09-05 11:35:55,110 Cashier #2: Ticket #02 is sold now
2018-09-05 11:35:55,110 Cashier #3: Solding out a ticket #03 ...
2018-09-05 11:35:55,118 Cashier #3: Ticket #03 is sold now
2018-09-05 11:35:55,118 Cashier #1: Solding out a ticket #04 ...
2018-09-05 11:35:55,125 Cashier #1: Ticket #04 is sold now
2018-09-05 11:35:55,125 Cashier #2: Solding out a ticket #05 ...
2018-09-05 11:35:55,132 Cashier #2: Ticket #05 is sold now
2018-09-05 11:35:55,132 Cashier #3: Solding out a ticket #06 ...
2018-09-05 11:35:55,139 Cashier #3: Ticket #06 is sold now
2018-09-05 11:35:55,139 Cashier #1: Solding out a ticket #07 ...
2018-09-05 11:35:55,146 Cashier #1: Ticket #07 is sold now
2018-09-05 11:35:55,146 Cashier #2: Solding out a ticket #08 ...
2018-09-05 11:35:55,153 Cashier #2: Ticket #08 is sold now
2018-09-

In [6]:

from threading import Condition

class Channel():

    def __init__(self):
        self.__data = None
        
    @property
    def ready(self):
        return self.__data is not None

    @property
    def data(self):
        d = self.__data
        self.__data = None
        return d
    
    @data.setter
    def data(self, data):
        self.__data = data
    

class Producer(Thread):
    def __init__(self, condition, channel):
        super().__init__()
        self.name="Producer Thread"
        self.__condition = condition
        self.__channel = channel

    
    def run(self):
        logger.info("Start")
        with self.__condition:
            self.__channel.data = "DATA 34-0563-0"
            self.__condition.notify()
            time.sleep(2)
    

class Consumer(Thread):
    def __init__(self, condition, channel):
        super().__init__()
        self.name="Consumer Thread"
        self.__condition= condition
        self.__channel = channel

    
    def run(self):
        logger.info("Start")
        with self.__condition:
            logger.info("Waiting for data ... ")
            cv.wait()
            logger.info("Consume DATA: %s" % self.__channel.data)


cv = Condition()
ch = Channel()

p = Producer(cv, ch)
c = Consumer(cv, ch)

c.start()
p.start()



2018-09-05 11:14:42,574 Consumer Thread: Start
2018-09-05 11:14:42,577 Producer Thread: Start
2018-09-05 11:14:42,577 Consumer Thread: Waiting for data ... 
2018-09-05 11:14:44,583 Consumer Thread: Consume DATA: DATA 34-0563-0


In [10]:
from threading import Condition

order_lock = Condition()
numbers = []

class NumbersThread(Thread):
    
    def __init__(self, name, condition, numbers, odd, highest):
        super().__init__(name=name)
        self.__condition = condition
        self.__numbers = numbers
        self.__number = 1 if odd else 2
        self.__highest = highest
    
    def run(self):
        logger.info("Starting ... ")
        while True:
            with self.__condition:            
                if not self.__number > self.__highest:
                    numbers.append(self.__number)
                    self.__number += 2
                else:
                    self.__condition.notify()
                    break
                    
                self.__condition.notify()
                self.__condition.wait()
                
        logger.info("Finished.")
        

class EvenNumbersThread(NumbersThread):
    
    def __init__(self, condition, numbers, highest):
        super().__init__(name="Even Numbers Thread", condition=condition, numbers=numbers, odd=False, highest=highest)
    

class OddNumbersThread(NumbersThread):
    
    def __init__(self, condition, numbers, highest):
        super().__init__(name="Odd Numbers Thread", condition=condition, numbers=numbers, odd=True, highest=highest)
        

ont = OddNumbersThread(condition=order_lock, numbers=numbers, highest=100)
ent = EvenNumbersThread(condition=order_lock, numbers=numbers, highest=100)

ont.start()
ent.start()

ent.join()

print(numbers)
    


2018-09-05 11:38:40,150 Odd Numbers Thread: Starting ... 
2018-09-05 11:38:40,153 Even Numbers Thread: Starting ... 
2018-09-05 11:38:40,160 Odd Numbers Thread: Finished.
2018-09-05 11:38:40,161 Even Numbers Thread: Finished.


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
